# Demo Setup

In [28]:
import connect
import requests
from automates.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet
from automates.gromet.query import query


We first set up access to the MIRA Epidemiology Domain Knowledge Graph (DKG) web service.

In [29]:
MIRA_DKG_URL = 'http://34.230.33.149:8771'

def get_mira_dkg_term(term, attribs):
    res = requests.get(MIRA_DKG_URL + '/api/search', params={'q': term})
    term = [entity for entity in res.json() if entity['id'].startswith('askemo')][0]
    res = {attrib: term.get(attrib) for attrib in attribs if term.get(attrib) is not None}
    return res

# Load CHIME SIR gromet representation

In [30]:
gromet_fn_module = json_to_gromet("gromet/CHIME_SIR_while_loop--Gromet-FN-auto.json")

The following uses the simple query to collect all named output ports of a GrometFNModule.
`nops` will hold a `List` of `Tuples`, where each tuple has the following format:
```
(<name_of_output_port>,    # named output port == a variable in source code
 <literal_value>,          # IF a literal value has been assigned, otherwise None
 <source_code_reference>)  # metadatum for the source code location of the assignment
```

In [31]:
nops = query.collect_named_output_ports(gromet_fn_module)
print(nops[0])

('inv_contact_rate', None, {'code_file_reference_uid': '09a64930-e9e4-8ae5-f825-bdf7c1e729f9',
 'col_begin': 4,
 'col_end': 50,
 'line_begin': 3,
 'line_end': 3,
 'metadata_type': 'source_code_reference',
 'provenance': {'method': 'skema_code2fn_program_analysis',
                'timestamp': '2022-11-30 16:18:33.012848'}})


# Load all the source, document, and other files we need

In [32]:
from IPython.display import IFrame

####
# HERE IS CONTENT FOR PART 2: FORMULA-TO-MODEL-MATCHING
####
# CHIME SVIIR SOURCE
CHIME_SVIIvR_model_source = connect.read_text_from_file("model/SVIIvR/CHIME_SVIIvR.py")

# Formula LaTeX source
CHIME_SVIIvR_FORMULAS_PDF_DOC = "model/SVIIvR/CHIME_SVIIvR_model_equations.pdf"
#formula_pdf_file = "model/SV"
CHIME_SVIIvR_FORMULAS_LATEX = connect.read_text_from_file("model/SVIIvR/formula.tex_idx")


####
# HERE IS CONTENT FOR PART 3: TEXT-TO-MODEL-MATCHING
####
CHIME_SIR_model_source = connect.read_text_from_file("model/SIR/CHIME_SIR_while_loop.py")
CHIME_SIR_DESCRIPTION_PDF_DOC = "model/SIR/CHIME-online-manual-T2021-01-19.pdf";
CHIME_SIR_DESCRIPTION_TEXT = connect.read_text_from_file("model/SIR/description.txt")

###
# HERE IS CONTENT FOR PART 4: BUCKY/DATA
###
BUCKY_model_source = connect.read_text_from_file("./model/Bucky/bucky_sample.py")



# Load Ontology 

Next, we choose a set of terms and attributes to pull into a local ontology that we can use to connect to the model. In addition, for some of the terms, we define custom ranges in which their values for this model can be adjusted.

In [33]:
# Terms we want to find in MIRA and specific attributes we want to add to our local ontology
terms = ['population', 'doubling time', 'recovery time', 'infectious time']
attribs = ['description', 'synonyms', 'xrefs', 'suggested_unit', 'suggested_data_type',
           'physical_min', 'physical_max', 'typical_min', 'typical_max']

# The local ontology if filled up from the MIRA DKG
LOCAL_ONTOLOGY = {
    term: get_mira_dkg_term(term, attribs) for term in terms
    }

# We can also set further local / use-case specific constraints as needed
LOCAL_ONTOLOGY['population']['typical_min'] = 1000
LOCAL_ONTOLOGY['population']['typical_max'] = 40_000_000

LOCAL_ONTOLOGY

{'population': {'description': 'The number of people who live in an area being modeled.',
  'synonyms': [],
  'xrefs': [{'id': 'ido:0000509', 'type': 'skos:exactMatch'}],
  'suggested_unit': 'person',
  'suggested_data_type': 'int',
  'physical_min': 0.0,
  'typical_min': 1000,
  'typical_max': 40000000},
 'doubling time': {'description': 'The length of time that an infectious disease requires to double in incidence.',
  'synonyms': [{'value': 'doubling rate',
    'type': 'oboInOwl:hasRelatedSynonym'}],
  'xrefs': [{'id': 'cemo:epidemic_doubling_time', 'type': 'skos:exactMatch'}],
  'suggested_unit': 'day',
  'suggested_data_type': 'float',
  'physical_min': 0.0},
 'recovery time': {'description': 'The length of time an infected individual needs to recover after being infected.',
  'synonyms': [{'value': 'mean recovery time',
    'type': 'oboInOwl:hasExactSynonym'}],
  'xrefs': [],
  'suggested_unit': 'day',
  'suggested_data_type': 'float',
  'physical_min': 0.0},
 'infectious time': 

# DEMO: Model-to-Resource Matching

# Demo Part 1: Ontology Matching

### 1a. Ontology to Gromet matching

In [7]:
targets = ['population', 'infectious time']
terms = list(LOCAL_ONTOLOGY.keys())

parameters = set()
var_dict = {}
for nop in nops:
    if nop[1] is not None:
        parameters.add(nop[0])
        var_dict[nop[0]] = nop

discoveredParameterConnections = connect.match_gromet_targets(targets, list(parameters), var_dict, terms)
discoveredParameterConnections

[('population', {'s_n': 'grometSubObject'}, 1000.0, 81),
 ('infectious time', {'i_n': 'grometSubObject'}, 1.0, 82)]

### 1b. Ontology to code matching

In [8]:
code = "model/SIR/CHIME_SIR_while_loop.py"
targets = ['population', 'infectious time']
discoveredParameterConnections = []
try:
    discoveredParameterConnections = connect.match_code_targets(targets, code, terms)
except OpenAIError as err:
    print("OpenAI connection error:", err)
    print("Using hard-coded connections")
    discoveredParameterConnections = [("infectious time", {"name": "grometSubObject"}, 14.0, 67),("population", {"name": "grometSubObject"}, 1000, 80)]

discoveredParameterConnections

Extracted variables:  [(3, 'inv_contact_rate', '1.0'), (11, 'growth_rate', '0'), (13, 'growth_rate', '2.0'), (34, 'index', '0'), (35, 'p_idx', '0'), (41, 'd_idx', '0'), (64, 'i_day', '17.0'), (65, 'n_days', '20'), (66, 'N_p', '3'), (67, 'N_t', '121'), (68, 'infections_days', '14.0'), (69, 'relative_contact_rate', '0.05'), (70, 'gamma', '1.0'), (81, 's_n', '1000'), (82, 'i_n', '1'), (83, 'r_n', '1'), (85, 'p_idx', '0')]


[('population', {'s_n': 'grometSubObject'}, 1000.0, 81),
 ('infectious time', {'infections_days': 'grometSubObject'}, 14.0, 68)]

# Demo Part 2: Formula-to-Model Matching

## We need the CHIME SVIIvR model...

In [9]:
print(CHIME_SVIIvR_model_source)


import sys
from csv import DictWriter, QUOTE_NONNUMERIC

def get_beta(intrinsic_growth_rate, gamma, susceptible, relative_contact_rate):
    """
    Calculates a rate of exposure given an intrinsic growth rate for COVID-19
    :param intrinsic_growth_rate: Rate of spread of COVID-19 cases
    :param gamma: The expected recovery rate from COVID-19 for infected individuals
    :param susceptible: Current amount of individuals that are susceptible
    :param relative_contact_rate: The relative contact rate amongst individuals in the population
    :return: beta: The rate of exposure of individuals to persons infected with COVID-19
    """
    inv_contact_rate = 1.0 - relative_contact_rate  # The inverse rate of contact between individuals in the population ## get_beta_icr_exp
    updated_growth_rate = intrinsic_growth_rate + gamma  # The intrinsic growth rate adjusted for the recovery rate from infection ## get_beta_ugr_exp
    beta = updated_growth_rate / susceptible * inv_contact_rate  

## And a document with some descriptive formulas... 

In [10]:
IFrame(CHIME_SVIIvR_FORMULAS_PDF_DOC, width=600, height=300)


## We used a tool to extract the source LaTeX from the PDF...

In [11]:
print(CHIME_SVIIvR_FORMULAS_LATEX)


1	S^\prime &=& - \beta S I - \beta S I_{v} - v_{r} S
2	V^\prime &=& v_{r} S - v_{s} V I - v_{s} V I_{v}
3	I^\prime &=& \beta S I + \beta S I_{v} - \gamma I
4	I_{v}^\prime &=& v_{s} V I + v_{s} V I_{v} - \gamma I_{v}
5	R^\prime &=& \gamma I + \gamma I_v



## Finally, we can find connections between the model and its descriptive formulas

In [12]:
connect.formula_code_connection(CHIME_SVIIvR_model_source, CHIME_SVIIvR_FORMULAS_LATEX)

The best match for the formula 1	S^\prime &=& - \beta S I - \beta S I_{v} - v_{r} S is the code section:

s_n = (
                      -beta * s * i - beta * s * i_v - vaccination_rate * s) + s  # Update to the amount of individuals that are susceptible ## sir_s_n_exp
---------------------------------------

The best match for the formula 2	V^\prime &=& v_{r} S - v_{s} V I - v_{s} V I_{v} is the following code section:

v_n = (vaccination_rate * s - beta * (1 - vaccine_efficacy) * v * i - beta * (
                1 - vaccine_efficacy) * v * i_v) + v  # Update to the amount of individuals that are susceptible ## sir_v_n_exp
---------------------------------------

The best match for the formula 3	I^\prime &=& \beta S I + \beta S I_{v} - \gamma I is the code section:

i_n = (
                      beta * s * i + beta * s * i_v - gamma_unvaccinated * i) + i  # Update to the amount of individuals that are infectious ## sir_i_n_exp
---------------------------------------

The best match fo

# Demo Part 3: Text-to-Model Matching

## We need the CHIME SIR model...


In [13]:
print(CHIME_SIR_model_source)

def get_beta(intrinsic_growth_rate, gamma,           
             susceptible, relative_contact_rate):    
    inv_contact_rate = 1.0 - relative_contact_rate  
    updated_growth_rate = intrinsic_growth_rate + gamma  
    beta = updated_growth_rate / susceptible * inv_contact_rate 
 
    return beta  

def get_growth_rate(doubling_time): 
    if doubling_time == 0:  
        growth_rate = 0  
    else:
        growth_rate = 2.0 ** (1.0 / doubling_time) - 1.0 
    return growth_rate 

def sir(s, i, r, beta, gamma, n): 
    s_n = (-beta * s * i) + s  
    i_n = (beta * s * i - gamma * i) + i 
    r_n = gamma * i + r  

    scale = n / (s_n + i_n + r_n) 

    s = s_n * scale 
    i = i_n * scale 
    r = r_n * scale 
    return s, i, r  

def sim_sir(s, i, r, gamma, i_day, 
            N_p, betas, days, 
            d_a, s_a, i_a, r_a, e_a  
            ):
    n = s + i + r 
    d = i_day 
    index = 0  
    p_idx = 0 

    while p_idx < N_p:  
        beta = betas[p_idx]  
        n_da

## And a PDF that describes the SIR model...

In [14]:
IFrame(CHIME_SIR_DESCRIPTION_PDF_DOC, width=600, height=300)


## We use a tool to extract the textual description of the SIR model from a few pages of the doc...

In [15]:
print(CHIME_SIR_DESCRIPTION_TEXT)

Discrete-time SIR modeling of infections/recovery
The model consists of individuals who are either Susceptible (S), Infected (I), or Recovered (R).
The epidemic proceeds via a growth and decline process. This is the core model of infectious disease spread and has been in use in epidemiology for many years.
The dynamics are given by the following 3 equations.
St+1 = St−βStIt
It+1 =It +βStIt−γIt
Rt+1 = Rt + γIt
To project the expected impact to Penn Medicine, we estimate the terms of the model.
To do this, we use a combination of estimates from other locations, informed estimates based on logical reasoning, and best guesses from the American Hospital Association.
Parameters
The model's parameters, β and γ , determine the severity of the epidemic. β can be interpreted as the effective contact rate: β=τ×c
which is the transmissibility τ multiplied by the average number of people exposed c. The transmissibility is the basic virulence of the pathogen. The number of people exposed c is the pa

## Finally, we can find connections between the model and descriptions from the text

In [16]:
connect.code_text_connection(CHIME_SIR_model_source, CHIME_SIR_DESCRIPTION_TEXT)

Best description for python function get_growth_rate is in lines 25-26:
	24	To estimate β directly, we'd need to know transmissibility and social contact rates. Since we don't know these things, we can extract it from known doubling times. The AHA says to expect a doubling time Td of 7-10 days. That means an early-phase rate of growth can be computed by using the doubling time formula:
>>	25	g = 21/Td −1
>>	26	Since the rate of new infections in the SIR model is g = βS − γ and we've already computed γ , β becomes
	27	a function of the initial population size of susceptible individuals β = (g + γ) . Initial Conditions
---------------------------------------
Best description for python function get_beta is in lines 11-12:
	10	The model's parameters, β and γ , determine the severity of the epidemic. β can be interpreted as the effective contact rate: β=τ×c
>>	11	which is the transmissibility τ multiplied by the average number of people exposed c. The transmissibility is the basic virulenc

# Demo Part 4: Data-to-Model Matching

## The Bucky model is large. Imagine the user has chosen some interesting functions...

In [17]:
print(BUCKY_model_source)

def estimate_cfr(
    g_data,
    base_CFR,
    case_to_death_time,
    Rh_gamma_k,
    S_age_dist,
    days_back=7,
):
    """Estimate CFR from recent case data."""

    mean = case_to_death_time  # params["H_TIME"] + params["I_TO_H_TIME"] #+ params["D_REPORT_TIME"]
    adm2_mean = xp.sum(S_age_dist * mean[..., None], axis=0)
    k = Rh_gamma_k

    rolling_case_hist = g_data.csse_data.incident_cases
    rolling_death_hist = g_data.csse_data.incident_deaths

    t_max = rolling_case_hist.shape[0]
    x = xp.arange(0.0, t_max)

    # adm0
    adm0_inc_cases = xp.sum(rolling_case_hist, axis=1)
    adm0_inc_deaths = xp.sum(rolling_death_hist, axis=1)

    adm0_theta = xp.sum(adm2_mean * g_data.Nj / g_data.N) / k

    w = 1.0 / (xp.special.gamma(k) * adm0_theta**k) * x ** (k - 1) * xp.exp(-x / adm0_theta)
    w = w / (1.0 - w)
    w = w / xp.sum(w)
    w = w[::-1]

    # n_loc = rolling_case_hist.shape[1]
    cfr = xp.empty((days_back,))
    for i in range(days_back):
        d = i + 1
  

## And maintains some interesting relevant datasets...

In [18]:
dataset_dir =  "./model/Bucky/data_sample/"
connect.print_df(dataset_dir)

We have a set of datasets:
---------------------------------------
Covid tracking data with 20781 rows x 56 columns. Schema and data sample:
         date state  positive  probableCases   negative  pending  \
0  2021-03-07    AK   56886.0            NaN        NaN      NaN   
1  2021-03-07    AL  499819.0       107742.0  1931711.0      NaN   

  totalTestResultsSource  totalTestResults  hospitalizedCurrently  \
0        totalTestsViral         1731628.0                   33.0   
1  totalTestsPeopleViral         2323788.0                  494.0   

   hospitalizedCumulative  ...  dataQualityGrade  deathIncrease  \
0                  1293.0  ...               NaN              0   
1                 45976.0  ...               NaN             -1   

   hospitalizedIncrease                                      hash  \
0                     0  dc4bccd4bb885349d7e94d6fed058e285d4be164   
1                     0  997207b430824ea40b8eb8506c19a93e07bc972e   

   commercialScore negativeRegularSc

## Finally, we can find connections between the functions and columns in these datasets

In [20]:
connect.code_dataset_connection(BUCKY_model_source,dataset_dir)

The most relevant columns to the function estimate_chr are the following:

-covid_tracking.csv: hospitalizations, deaths
-covid_confirmed_usafacts.csv: confirmed cases
-covid_deaths_usafacts.csv: deaths
-nychealth.csv: hospitalizations, deaths
-usafacts_hist.csv: confirmed cases, deaths
---------------------------------------
The most relevant columns to the estimate_cfr function are the following:

-case_to_death_time
-Rh_gamma_k
-S_age_dist
-days_back
-rolling_case_hist
-rolling_death_hist
-adm0_inc_cases
-adm0_inc_deaths
-adm0_theta
-w
-cfr
-adm0_cfr
-adm1_inc_cases
-adm1_inc_deaths
-adm1_theta
-baseline_adm1_cfr
-baseline_adm0_cfr
-cfr_fac
-adm0_cfr_fac
-valid
---------------------------------------
